# Week 2: Electric Power and Transformers

## Topics Covered:
- Ohm's Law (Review)
- Power in Electric Circuits
- Battery Ratings
- Basic Transformer
- Voltage Regulator
- AC-DC Conversion

## LAB 1: DC Power Supply Design

---

## Setup

In [ ]:
!pip install matplotlib numpy scipy schemdraw -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import schemdraw
import schemdraw.elements as elm
from scipy import signal

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Ohm's Law Review

$$V = I \times R$$

Where:
- V = Voltage (Volts)
- I = Current (Amperes)
- R = Resistance (Ohms)

## 2. Power in Electric Circuits

### Power Formula:
$$P = V \times I$$

### Derived Formulas:
Using Ohm's Law, we can derive:
- $$P = I^2 \times R$$ (Power dissipation in resistor)
- $$P = \frac{V^2}{R}$$ (Power from voltage and resistance)

Where:
- P = Power (Watts)
- Energy consumed per unit time

In [ ]:
def calculate_power(V=None, I=None, R=None):
    """
    Calculate power using various formulas
    Provide any two of V, I, or R
    """
    if V is not None and I is not None:
        P = V * I
        if R is None:
            R = V / I
    elif I is not None and R is not None:
        P = I**2 * R
        if V is None:
            V = I * R
    elif V is not None and R is not None:
        P = V**2 / R
        if I is None:
            I = V / R
    else:
        print("Error: Need at least two values")
        return None
    
    print(f"Voltage: {V:.3f}V")
    print(f"Current: {I*1000:.3f}mA ({I:.6f}A)")
    print(f"Resistance: {R:.3f}Ω")
    print(f"Power: {P:.6f}W ({P*1000:.3f}mW)")
    
    return {'V': V, 'I': I, 'R': R, 'P': P}

# Example: LED with current limiting resistor
print("Example 1: LED Circuit (9V battery, 330Ω resistor, 2V LED)")
print("Voltage across resistor: 7V")
result = calculate_power(V=7, R=330)

print("\n" + "="*50 + "\n")

print("Example 2: Power resistor (12V, 100Ω)")
calculate_power(V=12, R=100)

In [ ]:
# Visualize power dissipation
voltage_range = np.linspace(0, 12, 100)
resistances = [10, 50, 100, 500, 1000]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Power vs Voltage for different resistors
for R in resistances:
    power = voltage_range**2 / R
    ax1.plot(voltage_range, power, linewidth=2, label=f'R = {R}Ω')

ax1.set_xlabel('Voltage (V)', fontsize=12)
ax1.set_ylabel('Power (W)', fontsize=12)
ax1.set_title('Power Dissipation vs Voltage\n(P = V²/R)', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Power vs Current at fixed voltage
current = np.linspace(0, 1, 100)  # 0 to 1A
voltages = [5, 9, 12, 24]

for V in voltages:
    power = V * current
    ax2.plot(current * 1000, power, linewidth=2, label=f'V = {V}V')

ax2.set_xlabel('Current (mA)', fontsize=12)
ax2.set_ylabel('Power (W)', fontsize=12)
ax2.set_title('Power vs Current\n(P = V×I)', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Battery Ratings

### Key Battery Specifications:

**1. Voltage (V)**: Nominal voltage output
- Common: 1.5V (AA/AAA), 9V, 3.7V (Li-ion), 12V (car battery)

**2. Capacity (mAh or Ah)**: Total charge stored
- Milliampere-hours (mAh) or Ampere-hours (Ah)
- Example: 2000mAh battery can deliver 2000mA for 1 hour

**3. Energy (Wh)**: Total energy stored
$$Energy (Wh) = Voltage (V) \times Capacity (Ah)$$

**4. C-Rating**: Discharge rate
- 1C = Capacity in 1 hour
- 2C = Capacity in 0.5 hours (double the current)

In [ ]:
import pandas as pd

# Battery specifications table
battery_data = {
    'Type': ['AAA', 'AA', '9V', 'CR2032', '18650 Li-ion', 'Car Battery'],
    'Voltage (V)': [1.5, 1.5, 9.0, 3.0, 3.7, 12.0],
    'Capacity (mAh)': [1200, 2800, 600, 220, 3000, 50000],
    'Chemistry': ['Alkaline', 'Alkaline', 'Alkaline', 'Lithium', 'Li-ion', 'Lead-Acid']
}

df_battery = pd.DataFrame(battery_data)

# Calculate energy in Wh
df_battery['Energy (Wh)'] = df_battery['Voltage (V)'] * df_battery['Capacity (mAh)'] / 1000

print("Common Battery Specifications:")
print(df_battery.to_string(index=False))

# Visualize battery capacities
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Capacity comparison
colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold', 'plum', 'orange']
ax1.barh(df_battery['Type'], df_battery['Capacity (mAh)'], color=colors)
ax1.set_xlabel('Capacity (mAh)', fontsize=12)
ax1.set_title('Battery Capacity Comparison', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='x')

# Plot 2: Energy comparison
ax2.barh(df_battery['Type'], df_battery['Energy (Wh)'], color=colors)
ax2.set_xlabel('Energy (Wh)', fontsize=12)
ax2.set_title('Battery Energy Comparison', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

In [ ]:
def battery_runtime(capacity_mah, current_draw_ma):
    """
    Calculate battery runtime
    """
    runtime_hours = capacity_mah / current_draw_ma
    runtime_minutes = runtime_hours * 60
    
    print(f"Battery Capacity: {capacity_mah}mAh")
    print(f"Current Draw: {current_draw_ma}mA")
    print(f"Estimated Runtime: {runtime_hours:.2f} hours ({runtime_minutes:.1f} minutes)")
    
    # Account for efficiency (typically 80-90%)
    efficiency = 0.85
    actual_runtime_hours = runtime_hours * efficiency
    print(f"Practical Runtime (85% eff): {actual_runtime_hours:.2f} hours")
    
    return runtime_hours

print("Example 1: LED powered by AA battery")
battery_runtime(capacity_mah=2800, current_draw_ma=20)

print("\n" + "="*50 + "\n")

print("Example 2: Arduino powered by 9V battery")
battery_runtime(capacity_mah=600, current_draw_ma=50)

## 4. Basic Transformer

Transformers change AC voltage levels using electromagnetic induction.

### Transformer Equation:
$$\frac{V_s}{V_p} = \frac{N_s}{N_p} = \frac{I_p}{I_s}$$

Where:
- $V_p$ = Primary voltage
- $V_s$ = Secondary voltage
- $N_p$ = Primary turns
- $N_s$ = Secondary turns
- $I_p$ = Primary current
- $I_s$ = Secondary current

### Types:
- **Step-Down**: $V_s < V_p$ (More primary turns)
- **Step-Up**: $V_s > V_p$ (More secondary turns)
- **Isolation**: $V_s = V_p$ (Equal turns)

In [ ]:
def transformer_calculator(V_primary, N_primary, N_secondary, I_secondary=None):
    """
    Calculate transformer parameters
    """
    turns_ratio = N_secondary / N_primary
    V_secondary = V_primary * turns_ratio
    
    print(f"Primary Voltage: {V_primary}V")
    print(f"Primary Turns: {N_primary}")
    print(f"Secondary Turns: {N_secondary}")
    print(f"Turns Ratio: {turns_ratio:.3f}:1")
    print(f"Secondary Voltage: {V_secondary:.2f}V")
    
    if turns_ratio > 1:
        print(f"Type: Step-Up Transformer")
    elif turns_ratio < 1:
        print(f"Type: Step-Down Transformer")
    else:
        print(f"Type: Isolation Transformer")
    
    if I_secondary is not None:
        I_primary = I_secondary * turns_ratio
        print(f"\nSecondary Current: {I_secondary}A")
        print(f"Primary Current: {I_primary:.3f}A")
        print(f"Power (ideal): {V_secondary * I_secondary:.2f}W")
    
    return V_secondary

print("Example 1: Step-Down Transformer (120V to 12V)")
transformer_calculator(V_primary=120, N_primary=1000, N_secondary=100, I_secondary=2)

print("\n" + "="*50 + "\n")

print("Example 2: Step-Up Transformer (12V to 120V)")
transformer_calculator(V_primary=12, N_primary=100, N_secondary=1000)

In [ ]:
# Draw transformer circuit
with schemdraw.Drawing() as d:
    d.config(fontsize=12)
    
    # Primary side
    d += elm.SourceSin().label('120V\n60Hz')
    d += elm.Line().right(0.5)
    d += elm.Transformer(t1=10, t2=2).label('10:2')
    d += elm.Line().right(0.5)
    d += elm.Resistor().down().label('Load')
    d += elm.Line().left(3)

## 5. Voltage Regulator

Voltage regulators maintain constant output voltage despite variations in input voltage or load current.

### Types:
**Linear Regulators** (7805, LM317):
- Simple, low noise
- Inefficient (excess voltage dissipated as heat)
- Common: 7805 (5V), 7812 (12V), 7815 (15V)

**Switching Regulators** (Buck, Boost, Buck-Boost):
- High efficiency (80-95%)
- More complex, some noise
- Can step up or down voltage

In [ ]:
# Linear regulator efficiency calculation
def linear_regulator_efficiency(V_in, V_out, I_out):
    """
    Calculate linear regulator efficiency
    """
    P_out = V_out * I_out
    P_in = V_in * I_out  # Input current ≈ output current
    efficiency = (P_out / P_in) * 100
    P_dissipated = P_in - P_out
    
    print(f"Input Voltage: {V_in}V")
    print(f"Output Voltage: {V_out}V")
    print(f"Output Current: {I_out}A")
    print(f"\nOutput Power: {P_out:.3f}W")
    print(f"Input Power: {P_in:.3f}W")
    print(f"Power Dissipated (Heat): {P_dissipated:.3f}W")
    print(f"Efficiency: {efficiency:.1f}%")
    
    if P_dissipated > 1:
        print(f"\n⚠️ Warning: High heat dissipation! Use heatsink.")
    
    return efficiency

print("Example: 7805 Regulator (12V input, 5V output, 500mA load)")
linear_regulator_efficiency(V_in=12, V_out=5, I_out=0.5)

In [ ]:
# Compare linear vs switching regulator efficiency
V_in_range = np.linspace(7, 24, 100)
V_out = 5
I_out = 1  # 1A load

# Linear regulator efficiency
linear_eff = (V_out / V_in_range) * 100

# Switching regulator (typical ~85% efficiency)
switching_eff = np.ones_like(V_in_range) * 85

plt.figure(figsize=(12, 6))

plt.plot(V_in_range, linear_eff, 'r-', linewidth=2, label='Linear Regulator')
plt.plot(V_in_range, switching_eff, 'g-', linewidth=2, label='Switching Regulator')

plt.axhline(y=50, color='gray', linestyle='--', alpha=0.5, label='50% Efficiency')
plt.xlabel('Input Voltage (V)', fontsize=12)
plt.ylabel('Efficiency (%)', fontsize=12)
plt.title('Linear vs Switching Regulator Efficiency\n(5V output, 1A load)', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim(0, 100)

plt.tight_layout()
plt.show()

## 6. AC-DC Conversion (Rectification)

### Rectifier Types:

**1. Half-Wave Rectifier**
- Uses 1 diode
- Blocks negative half-cycle
- Low efficiency (~40%)

**2. Full-Wave Center-Tap Rectifier**
- Uses 2 diodes + center-tap transformer
- Both half-cycles used
- Better efficiency

**3. Full-Wave Bridge Rectifier**
- Uses 4 diodes
- Most efficient (~80%)
- No center-tap needed

In [ ]:
# Simulate rectifier circuits
time = np.linspace(0, 0.08, 1000)  # 80ms (4 cycles at 50Hz)
frequency = 50  # Hz
V_peak = 12 * np.sqrt(2)  # 12V RMS

# AC input
ac_input = V_peak * np.sin(2 * np.pi * frequency * time)

# Half-wave rectifier
half_wave = np.maximum(ac_input, 0)

# Full-wave rectifier
full_wave = np.abs(ac_input)

# Filtered DC (with smoothing capacitor)
# Simple approximation: exponential decay between peaks
filtered = full_wave.copy()
for i in range(1, len(filtered)):
    if filtered[i] < filtered[i-1] * 0.98:
        filtered[i] = filtered[i-1] * 0.995

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: AC Input
axes[0, 0].plot(time * 1000, ac_input, 'b-', linewidth=2)
axes[0, 0].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[0, 0].set_xlabel('Time (ms)', fontsize=11)
axes[0, 0].set_ylabel('Voltage (V)', fontsize=11)
axes[0, 0].set_title('AC Input (12V RMS, 50Hz)', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Half-Wave Rectifier
axes[0, 1].plot(time * 1000, half_wave, 'r-', linewidth=2)
axes[0, 1].fill_between(time * 1000, 0, half_wave, alpha=0.3)
axes[0, 1].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[0, 1].set_xlabel('Time (ms)', fontsize=11)
axes[0, 1].set_ylabel('Voltage (V)', fontsize=11)
axes[0, 1].set_title('Half-Wave Rectified', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Full-Wave Rectifier
axes[1, 0].plot(time * 1000, full_wave, 'g-', linewidth=2)
axes[1, 0].fill_between(time * 1000, 0, full_wave, alpha=0.3)
axes[1, 0].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[1, 0].set_xlabel('Time (ms)', fontsize=11)
axes[1, 0].set_ylabel('Voltage (V)', fontsize=11)
axes[1, 0].set_title('Full-Wave Rectified', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Filtered DC
axes[1, 1].plot(time * 1000, filtered, 'purple', linewidth=2)
axes[1, 1].fill_between(time * 1000, 0, filtered, alpha=0.3)
axes[1, 1].axhline(y=np.mean(filtered), color='orange', 
                    linestyle='--', linewidth=2, label=f'Average: {np.mean(filtered):.1f}V')
axes[1, 1].set_xlabel('Time (ms)', fontsize=11)
axes[1, 1].set_ylabel('Voltage (V)', fontsize=11)
axes[1, 1].set_title('Filtered DC (with Capacitor)', fontsize=12, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## LAB 1: DC Power Supply Design

### Objective:
Design a complete DC power supply that converts 120V AC to regulated 5V DC.

### Components:
1. **Transformer**: 120V AC → 12V AC
2. **Bridge Rectifier**: Convert AC to pulsating DC
3. **Filter Capacitor**: Smooth the DC
4. **Voltage Regulator**: 7805 (5V output)
5. **Output Capacitor**: Further filtering

### Design Steps:

In [ ]:
# DC Power Supply Design Calculator
def design_power_supply(V_ac_input, V_dc_output, I_max_output):
    """
    Design DC power supply components
    """
    print("DC POWER SUPPLY DESIGN")
    print("=" * 50)
    print(f"Input: {V_ac_input}V AC")
    print(f"Output: {V_dc_output}V DC @ {I_max_output}A max")
    print("\n" + "=" * 50)
    
    # 1. Transformer selection
    V_regulator_min = V_dc_output + 2  # Minimum dropout voltage
    V_ripple = 2  # Estimated ripple voltage
    V_diode_drop = 1.4  # Bridge rectifier drop (2 diodes)
    V_transformer_secondary = V_regulator_min + V_ripple + V_diode_drop
    
    print("\n1. TRANSFORMER")
    print(f"   Primary: {V_ac_input}V AC")
    print(f"   Secondary (minimum): {V_transformer_secondary:.1f}V AC RMS")
    print(f"   Recommended: 12V AC (standard)")
    print(f"   Power Rating: {V_transformer_secondary * I_max_output * 1.2:.1f}VA (with 20% margin)")
    
    # 2. Rectifier
    V_peak = V_transformer_secondary * np.sqrt(2)
    print(f"\n2. BRIDGE RECTIFIER")
    print(f"   Peak Voltage: {V_peak:.1f}V")
    print(f"   After diode drop: {V_peak - V_diode_drop:.1f}V")
    print(f"   Current Rating: {I_max_output * 1.5:.1f}A (with 50% margin)")
    print(f"   Recommended: 1N4007 diodes or W10 bridge module")
    
    # 3. Filter capacitor
    ripple_percent = 10  # 10% ripple
    frequency = 120  # Full-wave rectified frequency (2x mains)
    C_filter = (I_max_output * 1000) / (2 * frequency * V_peak * ripple_percent / 100)
    
    print(f"\n3. FILTER CAPACITOR")
    print(f"   Calculated: {C_filter:.0f}µF")
    print(f"   Recommended: {int(np.ceil(C_filter/1000)*1000)}µF (standard value)")
    print(f"   Voltage Rating: {V_peak * 1.5:.0f}V minimum (use 25V or 35V)")
    print(f"   Type: Electrolytic")
    
    # 4. Voltage regulator
    print(f"\n4. VOLTAGE REGULATOR")
    print(f"   Part: 78{V_dc_output:02d} (e.g., 7805 for 5V)")
    print(f"   Input Voltage Range: {V_dc_output + 2}V - {V_peak - V_diode_drop:.1f}V")
    print(f"   Output Current: {I_max_output}A")
    
    P_dissipated = (V_peak - V_diode_drop - V_dc_output) * I_max_output
    print(f"   Power Dissipation: {P_dissipated:.2f}W")
    
    if P_dissipated > 1:
        print(f"   ⚠️ HEATSINK REQUIRED!")
    
    # 5. Output capacitor
    print(f"\n5. OUTPUT CAPACITOR")
    print(f"   Value: 100µF (typical)")
    print(f"   Voltage: 10V minimum")
    print(f"   Type: Electrolytic")
    print(f"\n6. ADDITIONAL COMPONENTS")
    print(f"   - 0.1µF ceramic capacitor (noise filtering)")
    print(f"   - LED + 1kΩ resistor (power indicator)")
    print(f"   - Fuse: {I_max_output * 1.5:.1f}A on primary side")
    
    return {
        'transformer_secondary': 12,
        'filter_capacitor': int(np.ceil(C_filter/1000)*1000),
        'regulator': f'78{V_dc_output:02d}',
        'power_dissipation': P_dissipated
    }

# Design example: 5V, 1A power supply
result = design_power_supply(V_ac_input=120, V_dc_output=5, I_max_output=1)

In [ ]:
# Draw complete DC power supply schematic
with schemdraw.Drawing() as d:
    d.config(fontsize=10)
    
    # AC Input
    d += elm.SourceSin().label('120V AC')
    d += elm.Line().right(0.5)
    
    # Transformer
    d += elm.Transformer(t1=10, t2=1).label('120:12')
    d += elm.Line().right(0.5)
    
    # Bridge Rectifier (simplified)
    d += elm.Rect(w=1.5, h=1.5).label('Bridge\nRect')
    d += elm.Line().right(0.5)
    
    # Filter Capacitor
    d.push()
    d += elm.Capacitor().down().label('2200µF')
    d.pop()
    
    # Voltage Regulator (7805)
    d += elm.Line().right(0.5)
    d += elm.Rect(w=1.5, h=1).label('7805')
    d += elm.Line().right(0.5)
    
    # Output Capacitor
    d.push()
    d += elm.Capacitor().down().label('100µF')
    d.pop()
    
    # Output
    d += elm.Line().right(0.5)
    d += elm.Dot().label('5V DC', loc='right')

## Practice Exercises

### Exercise 1: Power Calculations
A 12V battery powers a 100Ω resistor. Calculate:
1. Current through resistor
2. Power dissipated
3. Energy consumed in 1 hour

In [ ]:
# Exercise 1 Solution
V = 12
R = 100

print("Exercise 1 Solution:")
result = calculate_power(V=V, R=R)

# Energy in 1 hour
time_hours = 1
energy_wh = result['P'] * time_hours
print(f"\nEnergy consumed in {time_hours} hour: {energy_wh:.3f}Wh")

### Exercise 2: Transformer Design
Design a transformer to step down 230V AC to 15V AC. If the primary has 1000 turns, how many secondary turns are needed?

In [ ]:
# Exercise 2 Solution
V_primary = 230
V_secondary_desired = 15
N_primary = 1000

# N_s = N_p * (V_s / V_p)
N_secondary = N_primary * (V_secondary_desired / V_primary)

print("Exercise 2 Solution:")
print(f"Primary: {V_primary}V, {N_primary} turns")
print(f"Secondary: {V_secondary_desired}V")
print(f"Secondary turns needed: {N_secondary:.0f} turns")

# Verify
transformer_calculator(V_primary=V_primary, N_primary=N_primary, 
                       N_secondary=int(N_secondary))

## Summary

This week we covered:

✅ **Ohm's Law Review**: V = I × R  
✅ **Power Calculations**: P = V × I = I²R = V²/R  
✅ **Battery Ratings**: Voltage, capacity (mAh), energy (Wh)  
✅ **Transformers**: Stepping AC voltage up or down  
✅ **Voltage Regulators**: Linear vs switching types  
✅ **AC-DC Conversion**: Rectification and filtering  
✅ **LAB 1**: Complete DC power supply design  

### Next Week Preview:
Week 3: Switches and Transistors - Active components that control circuits!
